In [ ]:
import os
from groq import Groq
import pandas as pd
import numpy as np

In [ ]:
apiKey = "fill_it_here"

In [ ]:
client = Groq(
    # api_key=os.environ.get("GROQ_API_KEY"),
    api_key= apiKey,
)

In [ ]:
dataset = pd.read_csv("keyphrases/done/keyphrases_NetflixLlama_Done.csv")

In [ ]:
dataset

In [ ]:
score_counts_balanced = dataset['sentiment'].value_counts().sort_index()
print(score_counts_balanced)

In [ ]:
train_data = dataset[['content', 'sentiment', 'Merged keyphrases_reduced 2']]
train_data['row_number'] = train_data.index

In [ ]:
train_data

In [ ]:
list = []
i = 0
for index, row in train_data.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    string_message = f"""I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - very negative, 2 - negative, 3 - neutral, 4 - positive, 5 - very positive. Here are examples to guide your classification:

                1. Comment: "I hate this App I tried it and tried it and tried it again but this app is not working what you doing now are you all updated it what is the problem is not even working.i even signed in it still did not show anything I am really annoyed pls try to fix it pls😠😠"
                Topics / Keyphrases: ['Netflix app freezing issue', 'Not showing content on app', 'Update problem with Netflix', 'Signed in but still not working', 'App not loading properly', 'Connection problem with Netflix', 'Error message on Netflix app', 'Fix Netflix app not working', 'Netflix app stuck on logo', 'Resolved Netflix app installation issues']
                Label: 1

                2. Comment: "Very bad. I think I got traumatised mentally physically. I don't think I can ever recover from This application causing me so much heartache from their bad bad shows and how bad their app is bad bad bad bad bad👎"
                Topics / Keyphrases: ['Netflix app problems', 'bad experience', 'traumatizing content', 'payment issues', 'access denied', 'favorite shows cancelled', 'graphic content ratings', 'disappointment and frustration', 'app causing pain', 'mental and physical trauma']
                Label: 1

                3. Comment: "You're better off paying 14.99 for the hulu and Disney bundle, Netflix out here being greedy and trying to charge me 22.99 for sharing my account with my sick dad who's on a fixed income (mind you after lying and saying they were going to charge for sharing accounts). Now of its just you and you wanna support this bs that's fair to each their own but I know I'm not and I know a lot of people who are not when they finally try to enforce it all the way."
                Topics / Keyphrases: ['Netflix subscription', 'content quality', 'price increase', 'app delay', 'US vs UK', 'ad-free', 'account sharing', 'content lag', 'streaming service complaints', 'corporate greed']
                Label: 1                

                4. Comment: "Netflix is a cancer you can't remove it from your tablet and it wastes my data by updating I have no control over my tablet I gave no permission for Netflix to install to my tablet I blame Samsung for allowing this in a country where they eat dogs I sure as hell won't be buying another tablet and phone from Samshit."
                Topics / Keyphrases: ['Netflix', 'Tablet', 'Data', 'Updating', 'Permission', 'Installation', 'Samsung', 'Foreign country', 'Android', 'Purchase decision']
                Label: 1

                5. Comment: "They keep raising their prices when 8 out of 10 of the shows they put out are just trash. Therefore, Netflix is a trash service now, I just wish everyone would cancel their memberships for a while to hurt Netflix's wallet and send a message."
                Topics / Keyphrases: ['Netflix prices too high', 'Lack of good shows', 'Poor content selection', 'Increasingly competitive market', 'High prices for low quality content', 'Netflix platform issues', 'Content removal is a problem', 'Disappointment with Netflix service', 'Loss of customers due to price increase', 'Need for better value from streaming service']
                Label: 1

                6. Comment: "I don't like the new mobile interface. I rate this app because I don't like the new interface, the old interface is already good, change is good but make sure that the interface update is more useful than this new one"
                Topics / Keyphrases: ['Netflix mobile app issues', 'Video streaming lags', 'Interface updates fail', 'Loading delays', 'Streaming errors', 'Playback issues', 'Connection problems', 'App crashes and freezes', 'Difficulty with login and profiles', 'Update and download difficulties']
                Label: 2

                7. Comment: "Good app but it's really annoying with the new Netflix household app. My dad owns the account, we both have two different houses and use different wifi networks on the same property, our houses are maybe a few thousand feet apart. And now I can't use Netflix without making another account, which we don't have the money for."
                Topics / Keyphrases: ['Netflix account sharing issues', 'Multiple households with limited account access', 'Login and travel limitations', 'Financial constraints for Netflix subscription', 'Alternative streaming services for unsatisfactory experience', 'Password sharing and restrictions', 'Limited show selection for family friendly content', 'Download limitations and parental control', 'Joint account management for multi-account streaming', 'Netflix customer dissatisfaction and content change']
                Label: 2
                                
                8. Comment: "Netflix always cancels a series. Why begin watching something that will be canceled? Netflix is such a disappointment. I only watch series on HBO and Amazon Prime at the moment."
                Topics / Keyphrases: ['Netflix issues', 'Show cancellation', 'Exit problems', 'Content dissatisfaction', 'Update issues', 'Subscription frustration', 'Navigation difficulties', 'Quality concerns', 'Availability problems', 'Account sharing disputes']
                Label: 2
                                                
                9. Comment: "Great movies at the beginning but now i can't watch them for free . I can only access it using a membership . why? I need answers from netflix"
                Topics / Keyphrases: ['netflix app issue', 'android download problem', 'login error', 'device limitation', 'netflix on ps4 issue', 'streaming service problem', 'connection error', 'content availability issue', 'app loading problem', 'payment method issue']
                Label: 2                

                10. Comment: "I have been a customer with Netflix since 2006 except when I was in the hospital for long stays. They used to be the best for movies,and shows even picking up shows like Lucifer after the network dropped it. But now it is the worst streaming service around. Not only are they charging more their programs are terrible, they get movies and shows that are 10 years old or older.due to their greed they keep charging more and more and now they are going to cut back. I will not renew my subscription"
                Topics / Keyphrases: ['Netflix subscription', 'content quality', 'price increase', 'app delay', 'US vs UK', 'ad-free', 'account sharing', 'content lag', 'streaming service complaints', 'corporate greed']
                Label: 2

                11. Comment: "Usually very good. There have been a few disappointments with some of your series. Overall I am happy with Netflix."
                Topics / Keyphrases: ['Netflix user experience', 'TV show quality', 'series reliability', 'positive app feedback', 'user satisfaction guarantee', 'content delight', 'reliable subscription service', 'good app functionality', 'series satisfaction', 'happiness with subscription service']
                Label: 3

                12. Comment: "The app is amazing and awesome!,but the new look is really disturbing to the eyes and once it freezes there's gonna be a big problem.Apart from that the app is wonderful."
                Topics / Keyphrases: ['Netflix app issues', 'Korean dramas on Netflix', 'Running text translation error', 'English audio glitch', 'Dubai Bling reality show bug', 'Netflix streaming problems', 'Smart TV WiFi issue', 'Netlix error fixes', 'App troubleshooting', 'Quality control bugs']
                Label: 3

                13. Comment: "Netflix just has too much space and all the apps I have are Snapchat, YouTube, and Netflix. No games. Plus Netflix keeps on doing a weird thing where I'm watching the episode and then it just goes straight to the next episode, to any of the Netflix staff members, please fix this issue, overall though I love it how I don't have to watch ads, so I'm rating it a 3 because it's not bad neither good, I hope that this was helpful to people that want to get Netflix :')"
                Topics / Keyphrases: ['Netflix availability issue', 'Movie not available', 'Removing popular content', 'Unofficial uploads', 'Missing episodes', 'Not available content', 'Quality of new releases', 'Content storage issues', 'Bass audio quality', 'Decreased content options']
                Label: 3

                14. Comment: "Ok not Fantastic mostly got it for Seinfeld which is the all time best Comedy.theres most made for TV shows. Not many Movies.No James Bond or classic Westerns. They used to show Rawhide and Rockford Files and some very entertaining independent Films. I also like Comedies with Bill Murray and Steve Martin and Chevy Chase you don't see their movies either. One thing they have on here I do enjoy is the Breaking Bad franchise and Bill Pullman Detective Series. For most part they don't show much new."
                Topics / Keyphrases: ['Netflix subscription model controversy', 'Disney influence on content', 'cancelled shows and fan disappointment', 'quality and family-friendly content concerns', 'subscription fatigue and delayed seasons', 'content release strategy and original programming', 'branded society and fascist faction flag', 'reality TV shows and user reviews', 'subscription rating and user frustration', 'positive message and original show diversity']
                Label: 3

                15. Comment: "App is good but completely overrides the brightness setting from the Android OS which is ridiculously annoying."
                Topics / Keyphrases: ['Netflix app issues', 'Brightness problem', 'Annoying feature', 'User experience', 'Poor implementation', 'Accessibility concern', 'Settings override', 'App design flaws', 'Streaming problem', 'Frustrating experience']
                Label: 3

                16. Comment: "In the Netflix app, you guys have a list of things that are "Coming Soon" etc. I think you should also have a list of series/movies that you'll be removing from the app or some kind of notification. THAT WILL BE GREAT IF IT HAPPENS."
                Topics / Keyphrases: ['Netflix App User Experience Improvement', 'Watch Show Suggestions and Feedback', 'Feature Request for Better Rating System', 'Improve Content Addition and Removal Notification', 'User Interface for Easier Content Discovery', 'Fast and Efficient Performance for Seamless Viewing', 'Provide More Options for User Feedback and Preference', 'Show Detailed Information for Better User Understanding', 'Remove Unwanted Content and Notify Users of Removal', 'Add More Content and Enhance User Experience with Feature Requests']
                Label: 4

                17. Comment: "1. Excellent, but upload more movies, series and episodes. 2. Put the video in super HD quality to enjoy watching. 3. Fixed the screen for samsung galaxy s23 ultra, to make it in full screen. 4. More setting for this apps. 5. Avatar should be upload by our choice."
                Topics / Keyphrases: ['Netflix app improvement', 'upload content options', '4k and super hd quality', 'easier navigation', 'app settings customization', 'profile and privacy concerns', 'multiple profile support', 'content upload interface', 'app updates and development', 'user experience optimization']
                Label: 4

                18. Comment: "(I'm a kid) Alright it's great like PERFECT but it has to be 9.7/10 PLZ ADD BOURTO NAURTO THE NEXT GENERATION! HULU HAS IT BUT THAT SERVICE DOG WATER"
                Topics / Keyphrases: ['Netflix streaming service issues', 'Anime episode removal', 'Content availability and removal', 'Membership and subscription problems', 'App features and functionality', 'Ratings and reviews', 'User experience and preferences', 'Streaming quality and lag', 'Favorite show disappointment', 'Alternative streaming site options']
                Label: 4

                19. Comment: "Netflix is an amazing app we can see non stop series and movies there is an one missing in Netflix is we all want to see language options that we automatically can chose language type movie most of movies are in other languages that we want see but because of language we can't see. Please see into this problem Thank you for this application"
                Topics / Keyphrases: ['netflix app issues', 'language options problem', 'episode availability issue', 'download problems', 'audio subtitles accuracy', 'movie series non availability', 'personalization and recommendation', 'premium content missing', 'payment and subscription issues', 'Customer service and support']
                Label: 4

                20. Comment: "we all like to watch stuff but i think that you can inprove your channels because somtimes it gets boring so i think that it would be beter for a wednesday 2,3 and for some more kid films that audilts over 20 (max) likes too because so ppl hate kid films (there ........ lets just say ......BORING) so i think you can make better kid films that parents will watch with there chidren and make sure there all entertaned with list time funny happy saddless bey :) :) :) :)"                
                Topics / Keyphrases: ['Netflix issues', 'Picture quality problems', 'Show cancellation concerns', 'Recommendation struggles', 'Kid-friendly content frustration', 'Adult audience dissatisfaction', 'Parenting concerns', 'Binge-watching disappointment', 'Show quality expectations', 'Streaming platform dissatisfaction']
                Label: 4

                21. Comment: "This app is one of the best app I've ever downloaded on my phone! It is a grate way of getting to watch all of your shows and relaxing after a day of work!!! Highly recommend getting the app!"
                Topics / Keyphrases: ['Netflix app', 'watch shows', 'movie streaming', 'highly recommend', 'movie selection', 'telemundo content', 'application quality', 'user satisfaction', 'streaming service quality', 'content availability']
                Label: 5

                22. Comment: "This is the best thing for TV shows and movies me andy mom love this app and it works so WELL they have some great ideas for movie and TV shows and it is just so cool five star worthy definitely Netflix is better that the Roku channel and I thought I would never say that. I love this app and everything in it thank you Netflix"
                Topics / Keyphrases: ['Netflix app', 'watch shows', 'movie streaming', 'highly recommend', 'movie selection', 'telemundo content', 'application quality', 'user satisfaction', 'streaming service quality', 'content availability']
                Label: 5

                23. Comment: "Love every thing about Netflix. Really enjoy the movies of all kinds, especially Christmas Movies..❤❤❤"
                Topics / Keyphrases: ['Netflix experience', 'Love watching movies', 'Christmas movie enjoyment', 'Netflix provides everything', 'Movie streaming experience', 'Enjoyable documentaries', 'Movies for everyone', 'Something for everyone on Netflix', 'Making daily entertainment', 'Daily movie streaming routine']
                Label: 5

                24. Comment: "THANKS NETFLIX !!! YOU ARE THE BEST FOR ADDING ONE PIECE WEEKLY ANIME, PLUS LIVE ACTION , PLUS SPINOFF , PLUS WIT REBOOT , THANK YOU NETFLIX , LOVE YOU 💕😭 , AGAIN THANKS FOR THIS VERY MUCH YOU ARE THE BEST STREAMING SERVICE AND I WILL CONTINUE TO SUPPORT YOU THROUGH BUYING YOUR PLANS ! JUST KEEP ONE PIECE GOING , THANKS AGAIN !!!! 💕"
                Topics / Keyphrases: ['Push notifications', 'Frequent notifications', 'Intrusive alerts', 'Notification overload', 'Notification spam', 'Netflix subscription cancellation', 'racially charged content', 'cultural representation issues', 'media portrayal concerns', 'customer dissatisfaction']
                Label: 5

                25. Comment: "I never have Netflix before but when my brother let me watch Netflix it was good! So I'm going to try it out now so Netflix is so so so so so! Good go and try it out now!!!!!!!:) (love precious ❤ 💖 💕 💗 💓 ♥)"
                Topics / Keyphrases: ['Netflix app', 'application', 'review', 'positive', 'entertainment', 'happy user', 'app enthusiasm', 'new update', 'update problems', 'difficult navigation']
                Label: 5

                Please provide the sentiment classification as a single number 
                (1, 2, 3, 4, or 5) and nothing more for the following comment:

                Comment: "{text}"
                Topics / Keyphrases: {topics}
                Label: ?
                """
    print("\n\n")
    print(string_message)
    print("\n")
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        model="llama3-8b-8192",
        # model="mixtral-8x7b-32768",
    )
    
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [ ]:
len(list)

In [ ]:
rest = train_data[len(list):]
for index, row in rest.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    string_message = f"""I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - very negative, 2 - negative, 3 - neutral, 4 - positive, 5 - very positive. Here are examples to guide your classification:

                1. Comment: "I hate this App I tried it and tried it and tried it again but this app is not working what you doing now are you all updated it what is the problem is not even working.i even signed in it still did not show anything I am really annoyed pls try to fix it pls😠😠"
                Topics / Keyphrases: ['Netflix app freezing issue', 'Not showing content on app', 'Update problem with Netflix', 'Signed in but still not working', 'App not loading properly', 'Connection problem with Netflix', 'Error message on Netflix app', 'Fix Netflix app not working', 'Netflix app stuck on logo', 'Resolved Netflix app installation issues']
                Label: 1

                2. Comment: "Very bad. I think I got traumatised mentally physically. I don't think I can ever recover from This application causing me so much heartache from their bad bad shows and how bad their app is bad bad bad bad bad👎"
                Topics / Keyphrases: ['Netflix app problems', 'bad experience', 'traumatizing content', 'payment issues', 'access denied', 'favorite shows cancelled', 'graphic content ratings', 'disappointment and frustration', 'app causing pain', 'mental and physical trauma']
                Label: 1

                3. Comment: "You're better off paying 14.99 for the hulu and Disney bundle, Netflix out here being greedy and trying to charge me 22.99 for sharing my account with my sick dad who's on a fixed income (mind you after lying and saying they were going to charge for sharing accounts). Now of its just you and you wanna support this bs that's fair to each their own but I know I'm not and I know a lot of people who are not when they finally try to enforce it all the way."
                Topics / Keyphrases: ['Netflix subscription', 'content quality', 'price increase', 'app delay', 'US vs UK', 'ad-free', 'account sharing', 'content lag', 'streaming service complaints', 'corporate greed']
                Label: 1                

                4. Comment: "Netflix is a cancer you can't remove it from your tablet and it wastes my data by updating I have no control over my tablet I gave no permission for Netflix to install to my tablet I blame Samsung for allowing this in a country where they eat dogs I sure as hell won't be buying another tablet and phone from Samshit."
                Topics / Keyphrases: ['Netflix', 'Tablet', 'Data', 'Updating', 'Permission', 'Installation', 'Samsung', 'Foreign country', 'Android', 'Purchase decision']
                Label: 1

                5. Comment: "They keep raising their prices when 8 out of 10 of the shows they put out are just trash. Therefore, Netflix is a trash service now, I just wish everyone would cancel their memberships for a while to hurt Netflix's wallet and send a message."
                Topics / Keyphrases: ['Netflix prices too high', 'Lack of good shows', 'Poor content selection', 'Increasingly competitive market', 'High prices for low quality content', 'Netflix platform issues', 'Content removal is a problem', 'Disappointment with Netflix service', 'Loss of customers due to price increase', 'Need for better value from streaming service']
                Label: 1

                6. Comment: "I don't like the new mobile interface. I rate this app because I don't like the new interface, the old interface is already good, change is good but make sure that the interface update is more useful than this new one"
                Topics / Keyphrases: ['Netflix mobile app issues', 'Video streaming lags', 'Interface updates fail', 'Loading delays', 'Streaming errors', 'Playback issues', 'Connection problems', 'App crashes and freezes', 'Difficulty with login and profiles', 'Update and download difficulties']
                Label: 2

                7. Comment: "Good app but it's really annoying with the new Netflix household app. My dad owns the account, we both have two different houses and use different wifi networks on the same property, our houses are maybe a few thousand feet apart. And now I can't use Netflix without making another account, which we don't have the money for."
                Topics / Keyphrases: ['Netflix account sharing issues', 'Multiple households with limited account access', 'Login and travel limitations', 'Financial constraints for Netflix subscription', 'Alternative streaming services for unsatisfactory experience', 'Password sharing and restrictions', 'Limited show selection for family friendly content', 'Download limitations and parental control', 'Joint account management for multi-account streaming', 'Netflix customer dissatisfaction and content change']
                Label: 2
                                
                8. Comment: "Netflix always cancels a series. Why begin watching something that will be canceled? Netflix is such a disappointment. I only watch series on HBO and Amazon Prime at the moment."
                Topics / Keyphrases: ['Netflix issues', 'Show cancellation', 'Exit problems', 'Content dissatisfaction', 'Update issues', 'Subscription frustration', 'Navigation difficulties', 'Quality concerns', 'Availability problems', 'Account sharing disputes']
                Label: 2
                                                
                9. Comment: "Great movies at the beginning but now i can't watch them for free . I can only access it using a membership . why? I need answers from netflix"
                Topics / Keyphrases: ['netflix app issue', 'android download problem', 'login error', 'device limitation', 'netflix on ps4 issue', 'streaming service problem', 'connection error', 'content availability issue', 'app loading problem', 'payment method issue']
                Label: 2                

                10. Comment: "I have been a customer with Netflix since 2006 except when I was in the hospital for long stays. They used to be the best for movies,and shows even picking up shows like Lucifer after the network dropped it. But now it is the worst streaming service around. Not only are they charging more their programs are terrible, they get movies and shows that are 10 years old or older.due to their greed they keep charging more and more and now they are going to cut back. I will not renew my subscription"
                Topics / Keyphrases: ['Netflix subscription', 'content quality', 'price increase', 'app delay', 'US vs UK', 'ad-free', 'account sharing', 'content lag', 'streaming service complaints', 'corporate greed']
                Label: 2

                11. Comment: "Usually very good. There have been a few disappointments with some of your series. Overall I am happy with Netflix."
                Topics / Keyphrases: ['Netflix user experience', 'TV show quality', 'series reliability', 'positive app feedback', 'user satisfaction guarantee', 'content delight', 'reliable subscription service', 'good app functionality', 'series satisfaction', 'happiness with subscription service']
                Label: 3

                12. Comment: "The app is amazing and awesome!,but the new look is really disturbing to the eyes and once it freezes there's gonna be a big problem.Apart from that the app is wonderful."
                Topics / Keyphrases: ['Netflix app issues', 'Korean dramas on Netflix', 'Running text translation error', 'English audio glitch', 'Dubai Bling reality show bug', 'Netflix streaming problems', 'Smart TV WiFi issue', 'Netlix error fixes', 'App troubleshooting', 'Quality control bugs']
                Label: 3

                13. Comment: "Netflix just has too much space and all the apps I have are Snapchat, YouTube, and Netflix. No games. Plus Netflix keeps on doing a weird thing where I'm watching the episode and then it just goes straight to the next episode, to any of the Netflix staff members, please fix this issue, overall though I love it how I don't have to watch ads, so I'm rating it a 3 because it's not bad neither good, I hope that this was helpful to people that want to get Netflix :')"
                Topics / Keyphrases: ['Netflix availability issue', 'Movie not available', 'Removing popular content', 'Unofficial uploads', 'Missing episodes', 'Not available content', 'Quality of new releases', 'Content storage issues', 'Bass audio quality', 'Decreased content options']
                Label: 3

                14. Comment: "Ok not Fantastic mostly got it for Seinfeld which is the all time best Comedy.theres most made for TV shows. Not many Movies.No James Bond or classic Westerns. They used to show Rawhide and Rockford Files and some very entertaining independent Films. I also like Comedies with Bill Murray and Steve Martin and Chevy Chase you don't see their movies either. One thing they have on here I do enjoy is the Breaking Bad franchise and Bill Pullman Detective Series. For most part they don't show much new."
                Topics / Keyphrases: ['Netflix subscription model controversy', 'Disney influence on content', 'cancelled shows and fan disappointment', 'quality and family-friendly content concerns', 'subscription fatigue and delayed seasons', 'content release strategy and original programming', 'branded society and fascist faction flag', 'reality TV shows and user reviews', 'subscription rating and user frustration', 'positive message and original show diversity']
                Label: 3

                15. Comment: "App is good but completely overrides the brightness setting from the Android OS which is ridiculously annoying."
                Topics / Keyphrases: ['Netflix app issues', 'Brightness problem', 'Annoying feature', 'User experience', 'Poor implementation', 'Accessibility concern', 'Settings override', 'App design flaws', 'Streaming problem', 'Frustrating experience']
                Label: 3

                16. Comment: "In the Netflix app, you guys have a list of things that are "Coming Soon" etc. I think you should also have a list of series/movies that you'll be removing from the app or some kind of notification. THAT WILL BE GREAT IF IT HAPPENS."
                Topics / Keyphrases: ['Netflix App User Experience Improvement', 'Watch Show Suggestions and Feedback', 'Feature Request for Better Rating System', 'Improve Content Addition and Removal Notification', 'User Interface for Easier Content Discovery', 'Fast and Efficient Performance for Seamless Viewing', 'Provide More Options for User Feedback and Preference', 'Show Detailed Information for Better User Understanding', 'Remove Unwanted Content and Notify Users of Removal', 'Add More Content and Enhance User Experience with Feature Requests']
                Label: 4

                17. Comment: "1. Excellent, but upload more movies, series and episodes. 2. Put the video in super HD quality to enjoy watching. 3. Fixed the screen for samsung galaxy s23 ultra, to make it in full screen. 4. More setting for this apps. 5. Avatar should be upload by our choice."
                Topics / Keyphrases: ['Netflix app improvement', 'upload content options', '4k and super hd quality', 'easier navigation', 'app settings customization', 'profile and privacy concerns', 'multiple profile support', 'content upload interface', 'app updates and development', 'user experience optimization']
                Label: 4

                18. Comment: "(I'm a kid) Alright it's great like PERFECT but it has to be 9.7/10 PLZ ADD BOURTO NAURTO THE NEXT GENERATION! HULU HAS IT BUT THAT SERVICE DOG WATER"
                Topics / Keyphrases: ['Netflix streaming service issues', 'Anime episode removal', 'Content availability and removal', 'Membership and subscription problems', 'App features and functionality', 'Ratings and reviews', 'User experience and preferences', 'Streaming quality and lag', 'Favorite show disappointment', 'Alternative streaming site options']
                Label: 4

                19. Comment: "Netflix is an amazing app we can see non stop series and movies there is an one missing in Netflix is we all want to see language options that we automatically can chose language type movie most of movies are in other languages that we want see but because of language we can't see. Please see into this problem Thank you for this application"
                Topics / Keyphrases: ['netflix app issues', 'language options problem', 'episode availability issue', 'download problems', 'audio subtitles accuracy', 'movie series non availability', 'personalization and recommendation', 'premium content missing', 'payment and subscription issues', 'Customer service and support']
                Label: 4

                20. Comment: "we all like to watch stuff but i think that you can inprove your channels because somtimes it gets boring so i think that it would be beter for a wednesday 2,3 and for some more kid films that audilts over 20 (max) likes too because so ppl hate kid films (there ........ lets just say ......BORING) so i think you can make better kid films that parents will watch with there chidren and make sure there all entertaned with list time funny happy saddless bey :) :) :) :)"                
                Topics / Keyphrases: ['Netflix issues', 'Picture quality problems', 'Show cancellation concerns', 'Recommendation struggles', 'Kid-friendly content frustration', 'Adult audience dissatisfaction', 'Parenting concerns', 'Binge-watching disappointment', 'Show quality expectations', 'Streaming platform dissatisfaction']
                Label: 4

                21. Comment: "This app is one of the best app I've ever downloaded on my phone! It is a grate way of getting to watch all of your shows and relaxing after a day of work!!! Highly recommend getting the app!"
                Topics / Keyphrases: ['Netflix app', 'watch shows', 'movie streaming', 'highly recommend', 'movie selection', 'telemundo content', 'application quality', 'user satisfaction', 'streaming service quality', 'content availability']
                Label: 5

                22. Comment: "This is the best thing for TV shows and movies me andy mom love this app and it works so WELL they have some great ideas for movie and TV shows and it is just so cool five star worthy definitely Netflix is better that the Roku channel and I thought I would never say that. I love this app and everything in it thank you Netflix"
                Topics / Keyphrases: ['Netflix app', 'watch shows', 'movie streaming', 'highly recommend', 'movie selection', 'telemundo content', 'application quality', 'user satisfaction', 'streaming service quality', 'content availability']
                Label: 5

                23. Comment: "Love every thing about Netflix. Really enjoy the movies of all kinds, especially Christmas Movies..❤❤❤"
                Topics / Keyphrases: ['Netflix experience', 'Love watching movies', 'Christmas movie enjoyment', 'Netflix provides everything', 'Movie streaming experience', 'Enjoyable documentaries', 'Movies for everyone', 'Something for everyone on Netflix', 'Making daily entertainment', 'Daily movie streaming routine']
                Label: 5

                24. Comment: "THANKS NETFLIX !!! YOU ARE THE BEST FOR ADDING ONE PIECE WEEKLY ANIME, PLUS LIVE ACTION , PLUS SPINOFF , PLUS WIT REBOOT , THANK YOU NETFLIX , LOVE YOU 💕😭 , AGAIN THANKS FOR THIS VERY MUCH YOU ARE THE BEST STREAMING SERVICE AND I WILL CONTINUE TO SUPPORT YOU THROUGH BUYING YOUR PLANS ! JUST KEEP ONE PIECE GOING , THANKS AGAIN !!!! 💕"
                Topics / Keyphrases: ['Push notifications', 'Frequent notifications', 'Intrusive alerts', 'Notification overload', 'Notification spam', 'Netflix subscription cancellation', 'racially charged content', 'cultural representation issues', 'media portrayal concerns', 'customer dissatisfaction']
                Label: 5

                25. Comment: "I never have Netflix before but when my brother let me watch Netflix it was good! So I'm going to try it out now so Netflix is so so so so so! Good go and try it out now!!!!!!!:) (love precious ❤ 💖 💕 💗 💓 ♥)"
                Topics / Keyphrases: ['Netflix app', 'application', 'review', 'positive', 'entertainment', 'happy user', 'app enthusiasm', 'new update', 'update problems', 'difficult navigation']
                Label: 5

                Please provide the sentiment classification as a single number 
                (1, 2, 3, 4, or 5) and nothing more for the following comment:

                Comment: "{text}"
                Topics / Keyphrases: {topics}
                Label: ?
                """
    print("\n\n")
    print(string_message)
    print("\n")
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        model="llama3-8b-8192",
        # model="mixtral-8x7b-32768",
    )
    
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [ ]:
len(list)

In [ ]:
# list.pop()

In [ ]:
list

In [ ]:
train_data['predictedSentiment'] = pd.Series(list)

In [ ]:
train_data

In [ ]:
import re

def contains_number(s):
    return any(char.isdigit() for char in s)

def findNumbers(string):
    # Find all numbers in the string
    numbers = None
    if(contains_number(string)):
        numbers = re.findall(r'\d+', string)
    else:
        numbers = ""
    
    # Convert the numbers to integers
    
    numbers = [int(x) for x in numbers]
    
    if(len(numbers)>0):
        x = numbers[0]-1
    else:
        x = -1
    return x

In [ ]:
formated = train_data['predictedSentiment'].apply(findNumbers)

In [ ]:
train_data['predictedSentiment'] = formated

In [ ]:
train_data

In [ ]:
train_data.to_csv('llamaOutput/topics/filenameNetflixFewShotLlama-T.csv', index=False)

In [ ]:
results = pd.read_csv("llamaOutput/topics/filenameNetflixFewShotLlama-T.csv")

In [ ]:
results

In [ ]:
results['predictedSentiment'].value_counts().sort_index()

In [ ]:
filtered_df = results[results["predictedSentiment"] == -1]

In [ ]:
filtered_df

In [ ]:
newResults = results[results["predictedSentiment"] != -1]

In [ ]:
newResults

In [ ]:
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def cm_analysis(y_true, y_pred, filename, labels, classes, ymap=None, figsize=(17,17)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      classes:   aliases for the labels. String array to be shown in the cm plot.
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    sns.set(font_scale=2.8)

    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.2f%%\n%d/%d' % (p, c, s)
            #elif c == 0:
            #    annot[i, j] = ''
            else:
                annot[i, j] = '%.2f%%\n%d' % (p, c)
    cm = confusion_matrix(y_true, y_pred, labels=labels, normalize='true')
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm = cm * 100
    cm.index.name = 'True Label'
    cm.columns.name = 'Predicted Label'
    fig, ax = plt.subplots(figsize=figsize)
    plt.yticks(va='center')

    sns.heatmap(cm, annot=annot, fmt='', ax=ax, xticklabels=classes, cbar=True, cbar_kws={'format':PercentFormatter()}, yticklabels=classes, cmap="Blues")
    plt.savefig(f'{filename}.png',  bbox_inches='tight')
    plt.savefig(f'{filename}.svg',  bbox_inches='tight')

In [ ]:
# Generate the confusion matrix
unique_labels = np.sort(newResults['scoreSA'].unique())
mapping = {0: 'Very Negative (1)', 1: 'Negative (2)', 2: 'Neutral (3)', 3: 'Positive (4)', 4: 'Very Positive (5)'}

# Applying mapping
classes = [mapping[value] for value in unique_labels]

In [ ]:
cm_analysis(newResults['scoreSA'], newResults['predictedSentiment'], 'API', unique_labels, classes)

In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support, mean_absolute_error

accuracy = accuracy_score(newResults['scoreSA'], newResults['predictedSentiment'])
precision_ALL = precision_score(newResults['scoreSA'], newResults['predictedSentiment'], average='macro')
recall_ALL = recall_score(newResults['scoreSA'], newResults['predictedSentiment'], average='macro')
f1_ALL = f1_score(newResults['scoreSA'], newResults['predictedSentiment'], average='macro')
mae_ALL = mean_absolute_error(newResults['scoreSA'], newResults['predictedSentiment'])
precision, recall, f1, support = precision_recall_fscore_support(newResults['scoreSA'], newResults['predictedSentiment'])

In [ ]:
# Calculate accuracy for each class
for i in range(len(unique_labels)):
    temp = newResults[newResults["scoreSA"] == i]
    value = accuracy_score(temp['scoreSA'], temp['predictedSentiment'])
    print(f'Label:\t{i}\t-\tValue:\t{value}')

In [ ]:
accuracy

In [ ]:
balanced_accuracy = balanced_accuracy_score(results['scoreSA'], results['predictedSentiment'])

In [ ]:
balanced_accuracy

In [ ]:
precision_ALL

In [ ]:
recall_ALL

In [ ]:
f1_ALL

In [ ]:
mae_ALL

In [ ]:
precision

In [ ]:
recall

In [ ]:
f1

In [ ]:
support